
# Predictive Maintenance (PdM) in Manufacturing 
<h3><span style="color: #117d30;"> Using Automated ML </span></h3>  



![](https://dreamdemostorageforgen2.blob.core.windows.net/mfgdemodata/PdM_Demo.jpg)


## Overview
*Dataset*: Telemetry data from 1000 machines **(8.6 Million events)** with reference data for machine failures, errors, past routine maintenance, and general information about machines. Time series features were extracted from telemetry data.

*Tools/Techniques*: AutoML 

### Notebook Organization 
+ Ingest featured manufacturing dataset

+ Create or use existing cluster

+ Prepare the model for deployment 

+ Submit the experiment to Azure ML to track the logs and metrics 

+ Build the AutoML model 

+ Deploy the best AutoML model as a Web Service 

+ Test the REST API and do inferencing 
















## Import the required libraries


In [1]:
import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig


## Configure workspace using credentials for Azure subscription

As part of the setup you have already created a Workspace. To run AutoML, you also need to create an Experiment. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.


In [2]:
from azureml.core import Workspace

# Importing user defined config
import config

# Import the subscription details as below to access the resources
subscription_id=config.subscription_id
resource_group=config.resource_group
workspace_name=config.workspace_name

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
ws.write_config()
ws = Workspace.from_config()

## Compute 

You will need to create a compute target for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.

Creation of AmlCompute takes approximately 5 minutes.
If the AmlCompute with that name is already in your workspace this code will skip the creation process. As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this article on the default limits and how to request more quota.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "mfgg-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Get Registered datasets 


In [5]:
from azureml.core.dataset import Dataset

train_data = Dataset.get_by_name(ws,"pdmmfg")

pdtrain_data = train_data.to_pandas_dataframe()

pdtrain_data.shape

(731358, 40)

## Preparing the data for model building


In [6]:
remove_columns = ['machineID','dt_truncated', 'failure','model_encoded','model']

pdtrain_data.drop(remove_columns, axis=1, inplace=True)

pdtrain_data.head(5)

,volt_rollingmean_12,rotate_rollingmean_12,pressure_rollingmean_12,vibration_rollingmean_12,volt_rollingmean_24,rotate_rollingmean_24,pressure_rollingmean_24,vibration_rollingmean_24,volt_rollingmean_36,vibration_rollingmean_36,...,error2sum_rollingmean_24,error3sum_rollingmean_24,error4sum_rollingmean_24,error5sum_rollingmean_24,comp1sum,comp2sum,comp3sum,comp4sum,age,label_e
0,166.950543,294.433319,94.473184,49.062098,165.197336,278.987299,97.318305,50.799015,166.522216,47.969229,...,0.0,0.0,0.0,0.0,489.0,549.0,549.0,564.0,18,0.0
1,165.290113,285.328277,96.147439,51.315016,164.807180,272.786127,99.193202,49.970419,167.168538,46.472198,...,0.0,0.0,0.0,0.0,489.0,549.0,549.0,564.0,18,0.0
2,164.324247,260.243976,102.238964,48.625823,168.107750,313.399517,102.155735,44.050788,170.913102,42.977346,...,0.0,0.0,0.0,0.0,488.0,548.0,548.0,563.0,18,0.0
3,171.891253,366.555058,102.072506,39.475754,174.207530,370.407544,101.847041,40.153107,171.536284,40.345945,...,0.0,0.0,0.0,0.0,488.0,548.0,548.0,563.0,18,0.0
4,176.523807,374.260029,101.621576,40.830461,171.358800,384.893390,98.324312,40.781041,168.852995,40.246936,...,0.0,0.0,0.0,0.0,487.0,547.0,547.0,562.0,18,0.0


## Printing the top 5 records in the y_train dataframe


In [7]:
from sklearn import datasets
from azureml.core.dataset import Dataset
from scipy import sparse
import os 
 
# Create a project_folder if it doesn't exist
if not os.path.isdir('data'):
 os.mkdir('data')
 
if not os.path.exists('project_folder'):
 os.makedirs('project_folder')
 
pdtrain_data.to_csv('./data/x_train.csv')
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='synapsemfgdata', overwrite=True, show_progress=True)
 
mfg_train = Dataset.Tabular.from_delimited_files(path=ds.path('synapsemfgdata/x_train.csv'))

Uploading an estimated of 1 files
Uploading ../data/x_train.csv
Uploaded ../data/x_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


#### Train the model - Please do not run the below step as it will take two hours to complete the model building.
## Set AutoML Configuration Parameters

The AutoMLConfig object defines the settings and data for an AutoML training job. Here, we set necessary inputs like the grain column name, the number of AutoML iterations to try, the training data, and cross-validation parameters.

It is generally recommended that users set forecast horizons to less than 100 time periods

Furthermore, AutoML's memory use and computation time increases in proportion to the length of the horizon, so consider carefully how this value is set. If a long horizon forecast really is necessary, consider aggregating the series to a coarser time scale.


In [12]:
import logging
from azureml.train.automl import AutoMLConfig

label_column_name = 'label_e' 

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_debuglog.log',
                             verbosity = logging.INFO,  
                             primary_metric='AUC_weighted',
                             iteration_timeout_minutes = 15,
                             experiment_timeout_hours=1, 
                             enable_early_stopping=True,
                             #featurization='auto', 
                             max_concurrent_iterations=2,
                             max_cores_per_iteration=-1,
                             enable_dnn=False,                             
                             n_cross_validations=2,                                                      
                             compute_target=compute_target, #amlcompute_target (to execute on aml cluster) 
                             #spark_context=sc, 
                             training_data=mfg_train,  
                             label_column_name='label_e',
                             #target_column_name='label_e',
                             enable_stack_ensemble=False,
                             enable_voting_ensemble=False,
                             momfgdel_explainability=True)

## Set Up 

As part of the setup you have already created a Workspace. To run AutoML, you also need to create an Experiment. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

## Experiment 
Call the submit method on the experiment object and pass the run configuration. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while. In this example, we specify show_output = True to print currently running iterations to the console. utomated ML runs more than 25 Machine Learning Algorithms and grades them according to performance.

## Please be aware that the below step will take around 2 hours to complete.






In [13]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "mfg_final_exp")
print(experiment)
local_run = experiment.submit(automl_config, show_output=True)

Experiment(Name: mfg_final_exp,
Workspace: Auto-ML-2)


WARNING - Received unrecognized parameter momfgdel_explainability


Running on remote or ADB.
Running on remote compute: mfgg-cluster
Parent Run ID: AutoML_de749252-6671-4c01-94da-a35dea2da072

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       DONE
DESCRIPTION:  A problem of class imbalance in your dataset has been detected and fixed.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+----

WARNING - Received unrecognized parameter momfgdel_explainability


         0   MaxAbsScaler LightGBM                          0:04:05       0.8352    0.8352


## Retrieve the best model 
Each run within an Experiment stores serialized (i.e. pickled) pipelines from the AutoML iterations. We can now retrieve the pipeline with the best performance on the validation dataset:


In [16]:
best_run, fitted_model = local_run.get_output() 

WARNING - Received unrecognized parameter momfgdel_explainability
WARNING - Received unrecognized parameter momfgdel_explainability
WARNING - The version of the SDK does not match the version the model was trained on.
WARNING - The consistency in the result may not be guaranteed.
WARNING - Package:azureml-automl-core, training version:1.10.0.post1, current version:1.9.0
Package:azureml-automl-runtime, training version:1.10.0.post1, current version:1.9.0
Package:azureml-core, training version:1.10.0, current version:1.9.0
Package:azureml-dataset-runtime, training version:1.10.0, current version:1.9.0
Package:azureml-defaults, training version:1.10.0, current version:1.9.0
Package:azureml-explain-model, training version:1.10.0, current version:1.9.0
Package:azureml-interpret, training version:1.10.0, current version:1.9.0
Package:azureml-pipeline-core, training version:1.10.0, current version:1.9.0
Package:azureml-telemetry, training version:1.10.0, current version:1.9.0
Package:azureml-

## Print the best run model


In [17]:
print(best_run)

Run(Experiment: mfg_final_exp,
Id: AutoML_de749252-6671-4c01-94da-a35dea2da072_0,
Type: azureml.scriptrun,
Status: Completed)


In [18]:
print(fitted_model)

Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...    silent=True, subsample=1.0, subsample_for_bin=200000,
          subsample_freq=0, verbose=-10))])
Y_transformer(['LabelEncoder', LabelEncoder()])


## Registering the model in Azure


In [20]:
#register model
description = "Mfg Classification Model"
model = local_run.register_model(description = description, tags={'Synapse': 'MFGModel'})
local_run.model_id

WARNING - Received unrecognized parameter momfgdel_explainability
WARNING - Received unrecognized parameter momfgdel_explainability


'AutoMLde74925260'

## Create scoring script
Create the scoring script, called score.py, used by the web service call to show how to use the model.


In [21]:
#saving scoring and conda file
script_file_name = 'inference/score.py'
conda_env_file_name = 'inference/env.yml'
#/content/azureml_automl.log
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'inference/env.yml')

## Specify package dependency for the environment.


In [22]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

conda = CondaDependencies.create(conda_packages=['numpy>=1.16.0,<=1.16.2','pandas','scikit-learn','py-xgboost<=0.80','fbprophet==0.5','psutil>=5.2.2,<6.0.0'],pip_packages=['azureml-defaults==1.0.83','azureml-train-automl-runtime==1.0.83.1','inference-schema','azureml-explain-model==1.0.83'])

myenv=Environment(name="automlenv")
myenv.python.conda_dependencies = conda

# Operationalize Model
Operationalization means getting the model into the cloud so that other can run it after you close the notebook. 

## Deploy the model as a Web Service on Azure Kubernetes Service 
##### (We have limited recources, hence please DO NOT RUN the below cells


In [23]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.compute import AksCompute
from azureml.core.model import Model

aks_name = 'new-aks'
aks_target = AksCompute(workspace=ws,name=aks_name)

aks_config = AksWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,auth_enabled=True)

inference_config = InferenceConfig(environment=myenv,
                                     entry_script = script_file_name)

api_service_name = 'mfg-realtime-pdm'

api_service = Model.deploy(workspace=ws,
                           name=api_service_name, 
                           models=[model],
                           inference_config=inference_config, 
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

api_service.wait_for_deployment(show_output=True)

Running........
Succeeded
AKS service creation operation finished, operation "Succeeded"


## Print logs for the API service


In [25]:
api_service.get_logs()

'2020-08-07T00:21:53,797365257+00:00 - iot-server/run \n2020-08-07T00:21:53,798512659+00:00 - rsyslog/run \n2020-08-07T00:21:53,798864659+00:00 - gunicorn/run \n2020-08-07T00:21:53,800213962+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_30c59c771a2c9019c4ce2fec8745f381/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_30c59c771a2c9019c4ce2fec8745f381/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_30c59c771a2c9019c4ce2fec8745f381/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_30c59c771a2c9019c4ce2fec8745f381/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_30c59c771a2c9019c4ce2fec8745f381/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

## Print the state of the deployed Web Service


In [26]:
print(api_service.state)

Healthy


## Inferencing using the REST API Endpoint
![](https://dreamdemostorageforgen2.blob.core.windows.net/mfgdemodata/PdM_Demo.jpg)


In [27]:
import urllib.request
import json
import os
import ssl
import pprint
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context
allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.
data = {"data":
        [
            [   
               406666,18,471.0,561.0, 471.0,501.0,0.0,0.0,0.0,0.0,0.0, 100.8627300454,99.521999589, 100.5907092691,1.3880873096,0.4344249378,0.3170432616, 467.5608723852,454.581998628,
               453.1206566629,4.3627672673,2.3310135872,2.2852285336,39.5870951038,39.1100578052,39.3093083177,0.50726284, 0.2221514412,0.1285150643,173.4308768104,171.2252424426, 169.9352949247,1.1288641558, 
               0.8682283269,0.8146674578
            ]
        ]               
}
body = str.encode(json.dumps(data))
url = api_service.scoring_uri 
print(url) # Replace the URL with your Endpoint URL
api_key = api_service.get_keys()[0] # Replace this with the API key for the web service
print(api_key)
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
req = urllib.request.Request(url, body, headers)
try:
    response = urllib.request.urlopen(req)

    result = response.read()
    #print(result)
    pprint.pprint(json.loads(result))
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

'{"result": [4.0]}'
